### Objetivo del Taller 02

Aplicar técnicas de limpieza, transformación y manejo estructurado de datos utilizando Spark DataFrames en Databricks, sobre un caso realista del negocio de un colegio.

Contexto y objetivo
Este taller evaluado consolida los aprendizajes de la Sesión 04 y 05, y promueve criterios de diseño basados en el negocio. El/la estudiante definirá y justificará las transformaciones según una industria de su elección (p. ej., retail, banca, seguros, telecom, farmacia, educación, logística), y construirá artefactos reproducibles en Databricks.

1. Importamos las dependencias

In [0]:
from pyspark.sql.functions import upper, col, trim, when, coalesce, lit, round, to_date, year, month, sum, countDistinct, avg
from pyspark.sql.types import DoubleType

2. creamos el catalogo y el esquema

In [0]:
catalog = "taller2"
schema_name = "colegios"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_name}")

3. ingestamos los archivos y definimos los dataframes

In [0]:
path_alumnos = "/Volumes/taller2/colegios/source/input/alumnos.json"
path_cursos = "/Volumes/taller2/colegios/source/input/cursos.json"
path_notas = "/Volumes/taller2/colegios/source/input/notas.json"
path_profesores = "/Volumes/taller2/colegios/source/input/profesores.json"

df_alumnos = spark.read.option("header", True).option("inferSchema", True).json(path_alumnos)
df_cursos = spark.read.option("header", True).option("inferSchema", True).json(path_cursos)
df_notas = spark.read.option("header", True).option("inferSchema", True).json(path_notas)
df_profesores = spark.read.option("header", True).option("inferSchema", True).json(path_profesores)

4. Limpiamos los datos

In [0]:
df_alumnos_clean = (
    df_alumnos
    .withColumn("ciudad", trim(upper(col("ciudad"))))
)

5. Hacemos Join con los demas dataframes

In [0]:
df_join_cursos = (
    df_cursos.alias("c")
    .join(
        df_profesores.alias("p").select("profesor_id", "nombre", "especialidad"),
        col("c.nombre_curso") == col("p.especialidad"),
        "inner"
    )
    .select(
        col("c.curso_id").alias("Curso_Id"),
        col("c.nombre_curso").alias("Curso_Nombre"),
        col("p.profesor_id").alias("Profesor_Id"),
        col("p.nombre").alias("Profesor")
    )
)

In [0]:
df_join = (
    df_notas.alias("n")
    .join(
        df_alumnos_clean.alias("ac").select("alumno_id", "nombre", "grado", "seccion"),
        col("n.alumno_id") == col("ac.alumno_id"),
        "inner"
    )
    .join(
        df_cursos.alias("c").select("curso_id", "nombre_curso"),
        col("n.curso_id") == col("c.curso_id"),
        "inner"
    )
    .join(
        df_join_cursos.alias("jc").select("Curso_Id", "Profesor"),
        col("n.curso_id") == col("jc.Curso_Id"),
        "inner"
    )
    .select(
        col("ac.alumno_id").alias("Alumno_Id"),
        col("ac.nombre").alias("Alumno_Nombre"),
        col("ac.grado").alias("Alumno_Grado"),
        col("ac.seccion").alias("Alumno_Seccion"),
        col("c.curso_id").alias("Curso_Id"),
        col("c.nombre_curso").alias("Curso_Nombre"),
        col("jc.Profesor").alias("Profesor"),
        col("n.nota").alias("Nota")
    )       
    
)

In [0]:
df_notas_procesada = (
    df_join
    .withColumn("notas_procesada", 
                when(col("Nota") >= 18, lit("Nota_Alta"))
                .when(col("Nota") >= 15, lit("Nota_Media"))
                .otherwise(lit("Nota_Baja"))
    )
)

In [0]:
tbl_detalle = f"{catalog}.{schema_name}.colegio_detalle"

In [0]:
df_join.write.format("delta").mode("overwrite").saveAsTable(tbl_detalle)

In [0]:
%sql
select * from taller2.colegios.colegio_detalle